# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 26 2022 9:17AM,242717,20,8169256-FLS-BO1,"ACI Healthcare USA, Inc.",Released
1,Jul 26 2022 9:17AM,242717,20,8305246,"ACI Healthcare USA, Inc.",Released
2,Jul 26 2022 9:17AM,242717,20,8308388,"ACI Healthcare USA, Inc.",Released
3,Jul 26 2022 9:17AM,242717,20,8319688,"ACI Healthcare USA, Inc.",Released
4,Jul 26 2022 9:17AM,242717,20,8321723,"ACI Healthcare USA, Inc.",Released
5,Jul 26 2022 9:17AM,242717,20,8319674,"ACI Healthcare USA, Inc.",Released
6,Jul 26 2022 9:17AM,242717,20,8319675,"ACI Healthcare USA, Inc.",Released
7,Jul 26 2022 9:17AM,242717,20,8319676,"ACI Healthcare USA, Inc.",Released
8,Jul 26 2022 9:17AM,242717,20,8319678,"ACI Healthcare USA, Inc.",Released
9,Jul 26 2022 9:17AM,242717,20,8319679,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,242718,Released,39
34,242719,Executing,8
35,242719,Released,1
36,242720,Released,50
37,242721,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242717,NaN,NaN,12.0
242718,NaN,NaN,39.0
242719,NaN,8.0,1.0
242720,NaN,NaN,50.0
242721,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242581,0.0,0.0,1.0
242612,0.0,0.0,34.0
242625,0.0,1.0,0.0
242632,40.0,0.0,23.0
242634,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242581,0,0,1
242612,0,0,34
242625,0,1,0
242632,40,0,23
242634,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242581,0,0,1
1,242612,0,0,34
2,242625,0,1,0
3,242632,40,0,23
4,242634,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242581,,,1
1,242612,,,34
2,242625,,1,
3,242632,40,,23
4,242634,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 26 2022 9:17AM,242717,20,"ACI Healthcare USA, Inc."
12,Jul 26 2022 9:12AM,242721,16,Sartorius Bioprocess Solutions
22,Jul 26 2022 9:12AM,242721,16,Sartorius Lab Products and Service
26,Jul 26 2022 9:03AM,242720,20,Alumier Labs Inc.
76,Jul 26 2022 9:02AM,242718,20,Alumier Labs Inc.
115,Jul 26 2022 8:58AM,242719,12,Hush Hush
124,Jul 26 2022 8:54AM,242714,20,"ACI Healthcare USA, Inc."
145,Jul 26 2022 8:42AM,242711,20,"ACI Healthcare USA, Inc."
170,Jul 26 2022 8:32AM,242716,10,ISDIN Corporation
189,Jul 26 2022 8:32AM,242713,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 26 2022 9:17AM,242717,20,"ACI Healthcare USA, Inc.",,,12
1,Jul 26 2022 9:12AM,242721,16,Sartorius Bioprocess Solutions,,,14
2,Jul 26 2022 9:12AM,242721,16,Sartorius Lab Products and Service,,,14
3,Jul 26 2022 9:03AM,242720,20,Alumier Labs Inc.,,,50
4,Jul 26 2022 9:02AM,242718,20,Alumier Labs Inc.,,,39
5,Jul 26 2022 8:58AM,242719,12,Hush Hush,,8,1
6,Jul 26 2022 8:54AM,242714,20,"ACI Healthcare USA, Inc.",,8,13
7,Jul 26 2022 8:42AM,242711,20,"ACI Healthcare USA, Inc.",,6,19
8,Jul 26 2022 8:32AM,242716,10,ISDIN Corporation,,,19
9,Jul 26 2022 8:32AM,242713,10,ISDIN Corporation,,,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 9:17AM,242717,20,"ACI Healthcare USA, Inc.",12,,
1,Jul 26 2022 9:12AM,242721,16,Sartorius Bioprocess Solutions,14,,
2,Jul 26 2022 9:12AM,242721,16,Sartorius Lab Products and Service,14,,
3,Jul 26 2022 9:03AM,242720,20,Alumier Labs Inc.,50,,
4,Jul 26 2022 9:02AM,242718,20,Alumier Labs Inc.,39,,
5,Jul 26 2022 8:58AM,242719,12,Hush Hush,1,8,
6,Jul 26 2022 8:54AM,242714,20,"ACI Healthcare USA, Inc.",13,8,
7,Jul 26 2022 8:42AM,242711,20,"ACI Healthcare USA, Inc.",19,6,
8,Jul 26 2022 8:32AM,242716,10,ISDIN Corporation,19,,
9,Jul 26 2022 8:32AM,242713,10,ISDIN Corporation,22,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 9:17AM,242717,20,"ACI Healthcare USA, Inc.",12,,
1,Jul 26 2022 9:12AM,242721,16,Sartorius Bioprocess Solutions,14,,
2,Jul 26 2022 9:12AM,242721,16,Sartorius Lab Products and Service,14,,
3,Jul 26 2022 9:03AM,242720,20,Alumier Labs Inc.,50,,
4,Jul 26 2022 9:02AM,242718,20,Alumier Labs Inc.,39,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 9:17AM,242717,20,"ACI Healthcare USA, Inc.",12.0,NaN,NaN
1,Jul 26 2022 9:12AM,242721,16,Sartorius Bioprocess Solutions,14.0,NaN,NaN
2,Jul 26 2022 9:12AM,242721,16,Sartorius Lab Products and Service,14.0,NaN,NaN
3,Jul 26 2022 9:03AM,242720,20,Alumier Labs Inc.,50.0,NaN,NaN
4,Jul 26 2022 9:02AM,242718,20,Alumier Labs Inc.,39.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 9:17AM,242717,20,"ACI Healthcare USA, Inc.",12.0,0.0,0.0
1,Jul 26 2022 9:12AM,242721,16,Sartorius Bioprocess Solutions,14.0,0.0,0.0
2,Jul 26 2022 9:12AM,242721,16,Sartorius Lab Products and Service,14.0,0.0,0.0
3,Jul 26 2022 9:03AM,242720,20,Alumier Labs Inc.,50.0,0.0,0.0
4,Jul 26 2022 9:02AM,242718,20,Alumier Labs Inc.,39.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3397380,211.0,6.0,1.0
12,242719,1.0,8.0,0.0
15,242632,23.0,0.0,40.0
16,728076,29.0,0.0,0.0
18,242685,0.0,1.0,0.0
19,728051,2.0,4.0,16.0
20,1456289,136.0,15.0,0.0
21,485411,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3397380,211.0,6.0,1.0
1,12,242719,1.0,8.0,0.0
2,15,242632,23.0,0.0,40.0
3,16,728076,29.0,0.0,0.0
4,18,242685,0.0,1.0,0.0
5,19,728051,2.0,4.0,16.0
6,20,1456289,136.0,15.0,0.0
7,21,485411,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,211.0,6.0,1.0
1,12,1.0,8.0,0.0
2,15,23.0,0.0,40.0
3,16,29.0,0.0,0.0
4,18,0.0,1.0,0.0
5,19,2.0,4.0,16.0
6,20,136.0,15.0,0.0
7,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,211.0
1,12,Released,1.0
2,15,Released,23.0
3,16,Released,29.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,1.0,0.0,40.0,0.0,0.0,16.0,0.0,0.0
Executing,6.0,8.0,0.0,0.0,1.0,4.0,15.0,0.0
Released,211.0,1.0,23.0,29.0,0.0,2.0,136.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,1.0,0.0,40.0,0.0,0.0,16.0,0.0,0.0
1,Executing,6.0,8.0,0.0,0.0,1.0,4.0,15.0,0.0
2,Released,211.0,1.0,23.0,29.0,0.0,2.0,136.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,1.0,0.0,40.0,0.0,0.0,16.0,0.0,0.0
1,Executing,6.0,8.0,0.0,0.0,1.0,4.0,15.0,0.0
2,Released,211.0,1.0,23.0,29.0,0.0,2.0,136.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()